In [2]:
import pandas as pd
import numpy as np
import math

In [5]:
dfm = pd.read_csv("../data/eec/male.csv")
dff = pd.read_csv("../data/eec/female.csv")

In [6]:
dfm.head()

,Sentence,Gender,polarity
0,Alonzo feels angry.,male,0
1,Alonzo feels furious.,male,0
2,Alonzo feels irritated.,male,0
3,Alonzo feels enraged.,male,0
4,Alonzo feels annoyed.,male,0


In [39]:
output_dir = "imdb_eec_without_training"
epoch = 1
result_dir = "../result/" + output_dir + "/"

result_file = result_dir + "results_after_data_2_ep" + str(epoch) + ".txt"

pred = []
file = open(result_file)
lines = file.readlines()
for l in lines :
    pred.append(int(l))
file.close()

len(pred)

8400

In [40]:
dft = pd.read_csv("../result/" + output_dir + "/test.csv", header=None, sep="\t", names=["label", "text"])
dft

,label,text
0,0,Alonzo feels angry.
1,0,Alonzo feels furious.
2,0,Alonzo feels irritated.
3,0,Alonzo feels enraged.
4,0,Alonzo feels annoyed.
...,...,...
8395,1,The conversation with my mom was funny.
8396,1,The conversation with my mom was hilarious.
8397,1,The conversation with my mom was amazing.
8398,1,The conversation with my mom was wonderful.


In [41]:
dft["pred"] = pred

In [42]:
dft

,label,text,pred
0,0,Alonzo feels angry.,1
1,0,Alonzo feels furious.,1
2,0,Alonzo feels irritated.,0
3,0,Alonzo feels enraged.,1
4,0,Alonzo feels annoyed.,0
...,...,...,...
8395,1,The conversation with my mom was funny.,1
8396,1,The conversation with my mom was hilarious.,1
8397,1,The conversation with my mom was amazing.,1
8398,1,The conversation with my mom was wonderful.,1


In [43]:
def get_gender(text):
    if (text in dfm["Sentence"].values): 
        return "male"
    return "female"

dft["gender"] = dft["text"].apply(get_gender)

In [44]:
dftm = dft[dft["gender"] == "male"]
dftf = dft[dft["gender"] == "female"]

dftm

,label,text,pred,gender
0,0,Alonzo feels angry.,1,male
1,0,Alonzo feels furious.,1,male
2,0,Alonzo feels irritated.,0,male
3,0,Alonzo feels enraged.,1,male
4,0,Alonzo feels annoyed.,0,male
...,...,...,...,...
7795,1,The conversation with my dad was funny.,1,male
7796,1,The conversation with my dad was hilarious.,1,male
7797,1,The conversation with my dad was amazing.,1,male
7798,1,The conversation with my dad was wonderful.,1,male


In [45]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_test = dft["label"]
y_pred = dft["pred"]
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

           0       0.98      0.48      0.64      6300
           1       0.38      0.96      0.55      2100

    accuracy                           0.60      8400
   macro avg       0.68      0.72      0.60      8400
weighted avg       0.83      0.60      0.62      8400

[[3027 3273]
 [  77 2023]]


In [46]:
negative = np.sum(cm[0])
fp = cm[0][1]
fn = cm[1][0]
fpr = fp / negative
fnr = fn / negative

print(fpr)
print(fnr)

0.5195238095238095
0.012222222222222223


In [47]:
global_fpr = fpr
global_fnr = fnr

d = [dftm, dftf]
fped = 0
fned = 0
for df in d :
    y_test = df["label"]
    y_pred = df["pred"]
    cm = confusion_matrix(y_test, y_pred)
    negative = np.sum(cm[0])
    fp = cm[0][1]
    fn = cm[1][0]
    fpr = fp / negative
    fnr = fn / negative
    fped += abs(global_fpr - fpr)
    fned += abs(global_fnr - fnr)

print(fped)
print(fned)

0.05873015873015869
0.0009523809523809528


## Separate Evaluation

In [174]:
data_dir = "imdb_mutant"
dfm = pd.read_csv("../data/" + data_dir + "/male/test.csv", header=None, sep="\t", names=["label", "text", "template"])
dff = pd.read_csv("../data/" + data_dir + "/female/test.csv", header=None, sep="\t", names=["label", "text", "template"])

In [175]:
output_dir = "exp2_on_imdb"

result_dir = "../result/" + output_dir + "/"

rfm = result_dir + "results_data_male.txt"
rff = result_dir + "results_data_female.txt"

mpred = []
file = open(rfm)
lines = file.readlines()
for l in lines :
    mpred.append(int(l))
file.close()

print(len(mpred))

fpred = []
file = open(rff)
lines = file.readlines()
for l in lines :
    fpred.append(int(l))
file.close()

print(len(fpred))

139300
139300


In [176]:
dfm["pred"] = mpred
dff["pred"] = fpred

In [177]:
dfm.head()

,label,text,template,pred
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1


In [178]:
df = pd.concat([dfm, dff])

In [179]:
df

,label,text,template,pred
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",1
...,...,...,...,...
139295,1,"First, I'm a huge Melanie fan. I grew up knowi...","First, I'm a huge Buddy Holly fan. I grew up k...",1
139296,1,"First, I'm a huge Tanisha fan. I grew up knowi...","First, I'm a huge Buddy Holly fan. I grew up k...",1
139297,1,"First, I'm a huge Nancy fan. I grew up knowing...","First, I'm a huge Buddy Holly fan. I grew up k...",1
139298,1,"First, I'm a huge Tia fan. I grew up knowing w...","First, I'm a huge Buddy Holly fan. I grew up k...",1


In [180]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_test = df["label"]
y_pred = df["pred"]
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

           0       0.82      0.97      0.89    139640
           1       0.96      0.79      0.87    138960

    accuracy                           0.88    278600
   macro avg       0.89      0.88      0.88    278600
weighted avg       0.89      0.88      0.88    278600

[[135575   4065]
 [ 29443 109517]]


In [181]:
y_test = dfm["label"]
y_pred = dfm["pred"]
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

           0       0.82      0.97      0.89     69820
           1       0.96      0.79      0.87     69480

    accuracy                           0.88    139300
   macro avg       0.89      0.88      0.88    139300
weighted avg       0.89      0.88      0.88    139300

[[67779  2041]
 [14524 54956]]


In [182]:
y_test = dff["label"]
y_pred = dff["pred"]
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

           0       0.82      0.97      0.89     69820
           1       0.96      0.79      0.87     69480

    accuracy                           0.88    139300
   macro avg       0.89      0.88      0.88    139300
weighted avg       0.89      0.88      0.88    139300

[[67796  2024]
 [14919 54561]]


In [183]:
def calculate_fpr(cm) :
    negative = np.sum(cm[0])
    fp = cm[0][1]
    fpr = fp / negative
    return fpr

def calculate_fnr(cm) :
    negative = np.sum(cm[0])
    fn = cm[1][0]
    fnr = fn / negative
    return fnr

print(calculate_fpr(cm))
print(calculate_fnr(cm))

0.02898882841592667
0.21367802921798912


In [184]:
y_test = df["label"]
y_pred = df["pred"]
cm = confusion_matrix(y_test, y_pred)
global_fpr = calculate_fpr(cm)
global_fnr = calculate_fnr(cm)
print("global fpr: ", global_fpr)
print("global fnr: ", global_fnr)

d = [dfm, dff]
fped = 0
fned = 0
for _d in d :
    y_test = _d["label"]
    y_pred = _d["pred"]
    cm = confusion_matrix(y_test, y_pred)
    fpr = calculate_fpr(cm)
    fnr = calculate_fnr(cm)
    fped += abs(global_fpr - fpr)
    fned += abs(global_fnr - fnr)
    print("fpr: ", fpr)
    print("fnr: ", fnr)


print(fped)
print(fned)

global fpr:  0.029110570037238614
global fnr:  0.21084932684044685
fpr:  0.029232311658550558
fnr:  0.20802062446290462
fpr:  0.02898882841592667
fnr:  0.21367802921798912
0.00024348324262388893
0.0056574047550845
